In [2]:
import csv,random,math,operator
def fract(x,y):
    if y==0:
        return 0
    return x/y

In [3]:
def loadcsv(filename):
    lines=csv.reader(open(filename,"r"))
    dataset=list(lines)
    for i in range(len(dataset)):
        dataset[i]=[float(x) for x in dataset[i]]
    return dataset

In [4]:
def splitDataset(dataset,splitRatio):
    trainSize=int(len(dataset)*splitRatio)
    print(trainSize)
    trainset=[]
    copy=list(dataset)
    while len(trainset)<trainSize:
        index=random.randrange(len(copy))
        trainset.append(copy.pop(index))
    return [trainset,copy]

In [5]:
def separateByClass(dataset):
    separated={}
    for i in range(len(dataset)):
        vector=dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]]=[]
        separated[vector[-1]].append(vector)
    return separated

In [6]:
def mean(numbers):
    return fract(sum(numbers),float(len(numbers)))
def stddev(numbers):
    avg=mean(numbers)
    variance=fract(sum([pow(x-avg,2)for x in numbers]),float(len(numbers)-1))
    return math.sqrt(variance)
def summerize(dataset):
    summeries=[(mean(attribute),stddev(attribute)) for attribute in zip(*dataset)]
    del summeries[-1]
    return summeries
def summerizeByClass(dataset):
    separated=separateByClass(dataset)
    summeries={}
    for classValue,instances in separated.items():
        summeries[classValue]=summerize(instances)
    return summeries

In [7]:
def calculateprobablity(x,mean,stddev):
    exponent=math.exp(-fract(math.pow(x-mean,2),(2*math.pow(stddev,2))))
    return fract(1,(math.sqrt(2*math.pi)*stddev))*exponent

In [9]:
def calculateclassprobablities(summeries,inputvector):
    probablities={}
    for classValue,classsummeries in summeries.items():
        probablities[classValue]=1
        for i in range(len(classsummeries)):
            mean,stddev=classsummeries[i]
            x=inputvector[i]
            print("**",mean,stddev,x)
            probablities[classValue]*=calculateprobablity(x,mean,stddev)
            print("individual probablity",probablities[classValue])
    return probablities

In [10]:
def predict(summeries,inputvector):
    probablities=calculateclassprobablities(summeries,inputvector)
    print("@@@@",probablities)
    bestLabel,bestProb=None,-1
    for classValue,probablity in probablities.items():
        if bestLabel is None or probablity>bestProb:
            bestProb=probablity
            bestLabel=classValue
    return bestLabel

def getpredictions(summeries,testset):
    predictions=[]
    for i in range(len(testset)):
        result=predict(summeries,testset[i])
        predictions.append(result)
    return predictions

def getaccuracy(testset,predictions):
    correct=0
    for i in range(len(testset)):
        if(testset[i][-1]==predictions[i]):
            correct+=1
    return fract(correct,float(len(testset)))*100.0

In [11]:
def NaiveBayes():
    filename='diabetes_6.csv'
    splitRatio=0.8
    dataset=loadcsv(filename)
    #print(dataset)
    
    print("\n The length of the Data set:",len(dataset))
    print("\n The Data set splitting into Training and testing\n")
    trainingset,testset=splitDataset(dataset,splitRatio)
    
    print("\n the no of rows in training set:{0} rows".format(len(trainingset)))
    
    print("\n the no of rows in testing set:{0} rows".format(len(testset)))
    
    print("\n the no of rows in training set:\n")
    for i in range(0,5):
        print(trainingset[i],"\n")
    print("\nfirst Five rows of testing set:\n")
    for i in range(0,3):
        print(testset[i],"\n")
    summeries=summerizeByClass(trainingset)
    print("\nModel Summerizes:\n",summeries)
    
    predictions=getpredictions(summeries,testset)
    print("\nPredictions:\n",predictions)
    
    accuracy=getaccuracy(testset,predictions)
    print("\nAccuracy:{0}%".format(accuracy))
NaiveBayes()


 The length of the Data set: 50

 The Data set splitting into Training and testing

40

 the no of rows in training set:40 rows

 the no of rows in testing set:10 rows

 the no of rows in training set:

[2.0, 141.0, 58.0, 34.0, 128.0, 25.4, 0.699, 24.0, 0.0] 

[2.0, 109.0, 92.0, 0.0, 0.0, 42.7, 0.845, 54.0, 0.0] 

[6.0, 144.0, 72.0, 27.0, 228.0, 33.9, 0.255, 40.0, 0.0] 

[4.0, 146.0, 85.0, 27.0, 100.0, 28.9, 0.189, 27.0, 0.0] 

[4.0, 129.0, 86.0, 20.0, 270.0, 35.1, 0.231, 23.0, 0.0] 


first Five rows of testing set:

[7.0, 187.0, 68.0, 39.0, 304.0, 37.7, 0.254, 41.0, 1.0] 

[0.0, 105.0, 64.0, 41.0, 142.0, 41.5, 0.173, 22.0, 0.0] 

[7.0, 114.0, 66.0, 0.0, 0.0, 32.8, 0.258, 42.0, 1.0] 


Model Summerizes:
 {0.0: [(3.28125, 2.8310988975558478), (99.9375, 32.075452879411614), (66.65625, 20.738944781380784), (20.8125, 15.582118577229437), (59.75, 87.83525634996755), (28.8125, 10.541247707259632), (0.45018749999999996, 0.33636668139820075), (29.875, 8.899981877473394)], 1.0: [(4.875, 5.249